## SILVER TO GOLD LAYER


In [1]:
import pandas as pd
import os
import pathlib

In [2]:
## Folder and File path
cwd = pathlib.Path(os.getcwd())

silver_data_path = cwd.joinpath("../DATA/SILVER").resolve()

gold_path = cwd.joinpath("../DATA/GOLD")
gold_path.mkdir(parents=True, exist_ok=True)

In [3]:
class TradeHistory:
    """Class Tread History"""

    def __init__(self, stock_name):
        self.stock_name = stock_name
        self.trade_price = []
        self.trade_quantity = []

    def fifo_sell_calc(self, trade_quantity):
        """Function to get fifo sell price"""
        old_self = self.trade_quantity.copy()
        for i, _ in enumerate(self.trade_price):
            if i == 0:
                self.trade_quantity[i] -= trade_quantity
            else:
                if self.trade_quantity[i - 1] < 0:
                    self.trade_quantity[i] += self.trade_quantity[i - 1]
                    self.trade_quantity[i - 1] = 0
                else:
                    break
        buy_price = 0
        for i, _ in enumerate(self.trade_quantity):
            buy_price += (old_self[i] - self.trade_quantity[i]) * self.trade_price[i]
        return buy_price / trade_quantity

    def holding_quantity(self):
        """Function to get Holding Quantity"""
        return sum(self.trade_quantity)

    def calc_avg_price(self):
        """Function to get avg price"""
        investment = 0
        for i, _ in enumerate(self.trade_quantity):
            investment += self.trade_quantity[i] * self.trade_price[i]
        if self.holding_quantity() != 0:
            return investment / self.holding_quantity()
        return 0

### Gold Layer - Trade History


In [4]:
# read the csv file
silver_data_trade_history_file_path = silver_data_path.joinpath("TradeHistory.json")


df_trade_history = pd.read_json(
    silver_data_trade_history_file_path, convert_dates=False
)


# Filter Equity trades


df_trade_history = df_trade_history[df_trade_history["segment"] == "EQ"]


df_trade_history["scrip_code"] = df_trade_history["scrip_code"].astype(int)


# replace scrip code to compnay name


silver_data_equity_file_path = silver_data_path.joinpath("Equity.json")


df_equity = pd.read_json(silver_data_equity_file_path)


# Merge df_trade_history with df_equity on the matching columns


df_trade_history = df_trade_history.merge(
    df_equity[["security_code", "security_id"]],
    left_on="scrip_code",
    right_on="security_code",
    how="left",
)


# Assign the new column 'stock_name' in df_trade_history to the values from 'security_id'


df_trade_history["stock_name"] = df_trade_history["security_id"]


# Combine 'date' and 'trade_time' into a single datetime column


# with fallback to 'date' if 'trade_time' is missing


# Convert the resulting column to datetime


df_trade_history["datetime"] = pd.to_datetime(
    df_trade_history["date"] + " " + df_trade_history["trade_time"].fillna("00:00:00"),
    format="%Y-%m-%d %H:%M:%S",
)


# sort the dataframe by date


df_trade_history = df_trade_history.sort_values(by="datetime")


df_trade_history["datetime"] = df_trade_history["datetime"].astype("str")


# rename the columns


df_trade_history = df_trade_history.rename(
    columns={
        "side": "trade_type",
        "quantity": "trade_quantity",
        "price": "trade_price",
    }
)


data_dict = list()


trade_history = dict()


for _, row in df_trade_history.iterrows():

    stock_name = row["stock_name"]

    if stock_name not in trade_history:

        trade_history[stock_name] = TradeHistory(stock_name)

    if row["trade_type"] == "Buy":

        row["buy_price"] = row["trade_price"]

        trade_history[stock_name].trade_price.append(row["trade_price"])

        trade_history[stock_name].trade_quantity.append(row["trade_quantity"])

    elif row["trade_type"] == "Sell":

        row["sell_price"] = row["trade_price"]

        row["buy_price"] = trade_history[stock_name].fifo_sell_calc(
            row["trade_quantity"]
        )

    else:

        raise Exception(f'{row["trade_type"]} was never excepected')

    row["holding_quantity"] = trade_history[stock_name].holding_quantity()

    row["avg_price"] = trade_history[stock_name].calc_avg_price()

    data_dict.append(row)


df_trade_history = pd.DataFrame(data_dict)


df_trade_history = df_trade_history.round(2)
df_trade_history = df_trade_history[
    [
        "datetime",
        "stock_name",
        "trade_type",
        "trade_quantity",
        "buy_price",
        "sell_price",
        "holding_quantity",
        "avg_price",
    ]
]


# Save the result as a json file


gold_trade_history_file_path = gold_path.joinpath("TradeHistory.json")


df_trade_history.to_json(gold_trade_history_file_path, orient="records", indent=4)


df_trade_history.info()


print("Gold Layer JSON file for trade history successfully created at:")


print(gold_trade_history_file_path.resolve())

<class 'pandas.core.frame.DataFrame'>
Index: 42 entries, 0 to 41
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   datetime          42 non-null     object 
 1   stock_name        42 non-null     object 
 2   trade_type        42 non-null     object 
 3   trade_quantity    42 non-null     float64
 4   buy_price         42 non-null     float64
 5   sell_price        15 non-null     float64
 6   holding_quantity  42 non-null     float64
 7   avg_price         42 non-null     float64
dtypes: float64(5), object(3)
memory usage: 4.0+ KB
Gold Layer JSON file for trade history successfully created at:
C:\Users\prashant.tripathi\Code\Unilever\Upstox\DATA\GOLD\TradeHistory.json
